# StudyFlow Assistant (Colab Runner)
Run this notebook from your school Chromebook. Everything executes inside Colab's VM.

**First-time setup:** Run cells 1-3 once. After that, just run cell 4 to start.

In [ ]:
#@title 1. Mount Google Drive & Copy Project
from google.colab import drive
drive.mount('/content/drive')

import os, shutil

DRIVE_PROJECT = '/content/drive/MyDrive/studyflow_assistant'
LOCAL_PROJECT = '/content/studyflow_assistant'

if not os.path.exists(DRIVE_PROJECT):
    print(f'ERROR: Upload the studyflow_assistant folder to Google Drive first!')
    print(f'Expected path: My Drive/studyflow_assistant/')
else:
    if os.path.exists(LOCAL_PROJECT):
        shutil.rmtree(LOCAL_PROJECT)
    shutil.copytree(DRIVE_PROJECT, LOCAL_PROJECT)
    print(f'Project copied to {LOCAL_PROJECT}')
    print(f'Files: {os.listdir(LOCAL_PROJECT)}')

In [ ]:
#@title 2. Install Dependencies
!pip install -q playwright==1.52.0 apscheduler==3.11.0 python-dotenv==1.1.0 tenacity==9.0.0 pydantic==2.11.3 pydantic-settings==2.9.1
!playwright install chromium
!playwright install-deps
print('All dependencies installed!')

In [ ]:
#@title 3. Set Up Environment (.env)
#@markdown Enter your Gemini API key below:
GEMINI_API_KEY = '' #@param {type:"string"}

env_path = '/content/studyflow_assistant/.env'
with open(env_path, 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
print(f'.env written to {env_path}')

# Verify
if not GEMINI_API_KEY:
    print('WARNING: No API key set. Drafts will fail.')

In [ ]:
#@title 4. First-Time Login (Manual)
#@markdown This opens a visible browser so you can sign into Google Classroom.
#@markdown Your session is saved for future headless runs.

import subprocess, os
os.chdir('/content/studyflow_assistant')

# Colab can't show a headed browser directly, so we use a VNC approach
# Install VNC + noVNC for browser visibility
!apt-get update -qq && apt-get install -y -qq xvfb x11vnc novnc > /dev/null 2>&1

# Start virtual display
!Xvfb :99 -screen 0 1280x720x24 &
import os
os.environ['DISPLAY'] = ':99'

# Start noVNC so you can see the browser in a new tab
!x11vnc -display :99 -forever -nopw -shared -rfbport 5900 &
!cd /usr/share/novnc && ./utils/novnc_proxy --vnc localhost:5900 --listen 6080 &

import time
time.sleep(3)

from google.colab.output import eval_js
url = eval_js("google.colab.kernel.proxyPort(6080)")
print(f'\n=== BROWSER VIEW ===')
print(f'Open this link in a new tab to see the browser:')
print(url)
print(f'Then come back here and run the next cell.\n')

In [ ]:
#@title 5. Sign In to Google Classroom
#@markdown After opening the VNC link above, run this cell.
#@markdown Watch the browser in the VNC tab and sign in when prompted.

import asyncio, os, sys
os.chdir('/content/studyflow_assistant')
sys.path.insert(0, '/content/studyflow_assistant')
os.environ['STUDYFLOW_MODE'] = 'run'
os.environ['DISPLAY'] = ':99'

async def manual_login():
    from playwright.async_api import async_playwright
    pw = await async_playwright().start()
    ctx = await pw.chromium.launch_persistent_context(
        user_data_dir='/content/studyflow_assistant/.browser_data',
        headless=False,
        args=['--no-sandbox', '--disable-dev-shm-usage'],
        viewport={'width': 1280, 'height': 720},
    )
    page = ctx.pages[0] if ctx.pages else await ctx.new_page()
    await page.goto('https://accounts.google.com/ServiceLogin')
    print('Browser opened to Google sign-in.')
    print('Go to the VNC tab and sign in with your school Google account.')
    print('After signing in, navigate to classroom.google.com.')
    print('Once you see your classes, come back here and press STOP on this cell.')
    # Keep alive until user stops the cell
    while True:
        await asyncio.sleep(5)

try:
    await manual_login()
except (KeyboardInterrupt, asyncio.CancelledError):
    print('Login session saved! You can now run StudyFlow headlessly.')

In [ ]:
#@title 6. Run StudyFlow (Single Run)
#@markdown Scans assignments, generates drafts, pastes into Google Docs.

import subprocess, os
os.chdir('/content/studyflow_assistant')
os.environ['DISPLAY'] = ':99'

result = subprocess.run(
    ['python', 'main.py', 'run'],
    capture_output=False,
    text=True,
    cwd='/content/studyflow_assistant',
    env={**os.environ, 'STUDYFLOW_MODE': 'run'},
)
print(f'\nExit code: {result.returncode}')

In [ ]:
#@title 7. Save Session Back to Drive
#@markdown Copies browser session + drafts back to Drive so they persist.

import shutil, os

DRIVE_PROJECT = '/content/drive/MyDrive/studyflow_assistant'
LOCAL_PROJECT = '/content/studyflow_assistant'

# Copy browser data and drafts back to Drive
for folder in ['.browser_data', 'drafts', 'downloads']:
    src = os.path.join(LOCAL_PROJECT, folder)
    dst = os.path.join(DRIVE_PROJECT, folder)
    if os.path.exists(src):
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f'Saved {folder} to Drive')

print('Done! Your session and drafts are saved to Google Drive.')